In [3]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.8 MB/s eta 0:00:00


In [4]:
!brew install portaudio

/bin/bash: line 1: brew: command not found


In [5]:
! apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libjack-dev libjack0
Suggested packages:
  libasound2-doc jackd1 portaudio19-doc
The following packages will be REMOVED:
  libjack-jackd2-0
The following NEW packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
  portaudio19-dev
0 upgraded, 6 newly installed, 1 to remove and 41 not upgraded.
Need to get 596 kB of archives.
After this operation, 3,178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjack0 amd64 1:0.125.0-3build2 [93.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libasound2-dev amd64 1.2.6.1-1ubuntu1 [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjack-dev amd64 1:0.125.0-3build2 [206 kB]
Get:4 http://archive.ubuntu.com/u

In [13]:
! pip install pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp312-cp312-linux_x86_64.whl size=68673 sha256=d8629406e45091635a972d7af10df7882a159e3ca4731a45b9800cea20d22c35
  Stored in directory: /root/.cache/pip/wheels/68/c7/33/c6a6b210cb5819ec5c219928c794a447742a7d86d21c0b92e6
Successfully built pyaudio


In [14]:
!pip install vosk

In [7]:
! pip install transformers

In [8]:
!pip install torch

In [10]:
!pip install transformers accelerate datasets scipy torch

In [11]:
import ipywidgets as widgets
from IPython.display import display
from queue import Queue
from threading import Thread
import base64
from google.colab import output
import io
import wave
from transformers import pipeline
import torch
import time # Import time for adding a delay

# --- Queues and Audio Configuration ---
messages = Queue()
recordings = Queue()
SAMPLE_RATE = 16000

# Determine the device (GPU if available, otherwise CPU)
device_id = 0 if torch.cuda.is_available() else -1

# Initialize Whisper for speech recognition
transcriber = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-tiny.en",
    device=device_id
)


def record_microphone():
    """Starts and manages the JavaScript microphone recording utility."""
    JS_CODE = f"""
        const div = document.createElement('div');
        document.body.appendChild(div);

        const script = document.createElement('script');
        script.src = 'https://cdn.rawgit.com/Miguelao/recorderjs/master/recorder.js';
        div.appendChild(script);

        return new Promise(resolve => {{
            script.onload = () => {{
                navigator.mediaDevices.getUserMedia({{
                    audio: {{
                        sampleRate: {SAMPLE_RATE},
                        channelCount: 1
                    }}
                }})
                    .then(stream => {{
                        const recorder = new Recorder(stream, {{
                            sampleRate: {SAMPLE_RATE}
                        }});
                        recorder.record();

                        // Record for 3 seconds per chunk
                        setTimeout(() => {{
                            recorder.stop();
                            stream.getTracks().forEach(track => track.stop());

                            recorder.exportWAV(blob => {{
                                const reader = new FileReader();
                                reader.onloadend = () => {{
                                    resolve(reader.result.split(',')[1]);
                                }};
                                reader.readAsDataURL(blob);
                            }});
                        }}, 3000);
                    }});
            }};
        }});
    """

    while messages.empty() is True:
        try:
            audio_base64 = output.eval_js(JS_CODE)
            recordings.put(audio_base64)
        except Exception:
            break

def speech_recognition(output_widget):
    """Retrieves recorded audio, saves it as WAV, and transcribes using Whisper."""

    with output_widget:
        display(widgets.HTML(value="<p style='color:blue;'>**Awaiting first audio chunk...**</p>"))

    all_audio_data = []

    while messages.empty() is True or not recordings.empty():
        try:
            audio_base64 = recordings.get(timeout=1.0)
            audio_bytes = base64.b64decode(audio_base64)

            with io.BytesIO(audio_bytes) as wav_file:
                wav_reader = wave.open(wav_file, 'rb')
                all_audio_data.append(wav_reader.readframes(wav_reader.getnframes()))
                wav_reader.close()

            with output_widget:
                display(widgets.HTML(value="<p style='color:blue;'>*Received chunk.*</p>"))

            recordings.task_done()

        except Exception:
            pass

    # --- Transcription Phase ---

    with output_widget:
        display(widgets.HTML(value="<hr><h3>🎙️ **Transcribing Audio...**</h3>"))

    if not all_audio_data:
        with output_widget:
            display("No audio data was captured. Please ensure microphone permission is granted.")
        return

    combined_audio = b"".join(all_audio_data)

    with wave.open('temp_recording.wav', 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(SAMPLE_RATE)
        wf.writeframes(combined_audio)

    try:
        transcription_result = transcriber("temp_recording.wav")
        final_text = transcription_result['text']

        with output_widget:
            display(widgets.HTML(value=f"<p style='color:green;'>✅ **Transcription Complete:**</p>"))
            display(final_text)

    except Exception as e:
        with output_widget:
            display(f"Transcription Error: {e}")
    finally:
         with output_widget:
            display(widgets.HTML(value=f"<hr>Total Audio Data Processed: **{len(combined_audio) / 1024:.2f} KB**"))


# --- Widget Setup ---

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=True,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

output_widget = widgets.Output()

def start_recording(data):
    record_button.disabled = True
    stop_button.disabled = False
    output_widget.clear_output()

    messages.put(True)

    with output_widget:
        display(widgets.HTML(value="<p style='color:green;'>**Starting Recording... (Grant microphone access in browser)**</p>"))

    record = Thread(target=record_microphone)
    record.start()

    # Add a short delay to allow JS to initialize before the transcription thread starts
    time.sleep(1)

    transcribe = Thread(target=speech_recognition, args=(output_widget,))
    transcribe.start()

def stop_recording(data):
    stop_button.disabled = True
    record_button.disabled = False

    with output_widget:
        if not messages.empty():
            messages.get()
        display(widgets.HTML(value="<p style='color:orange;'>**Stopping Recording. Please wait for transcription...**</p>"))


record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output_widget)

Device set to use cpu


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', disabled=True, icon='stop', style=ButtonStyle(), tooltip='S…

Output()

In [15]:
# Install pyaudio from http://people.csail.mit.edu/hubert/pyaudio/
# Find audio device index using this code
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

In [16]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=2,
                    frames_per_buffer=chunk)

    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [17]:
import subprocess
import json
from vosk import Model, KaldiRecognizer
import time

model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):

    while not messages.empty():
        frames = recordings.get()

        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]

        cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)
        output.append_stdout(cased)
        time.sleep(1)

vosk-model-en-us-0.22.zip: 100%|██████████| 1.78G/1.78G [01:45<00:00, 18.2MB/s]
